# 🌟 ¡Nuestra Receta Musical con el Código final para nuestro modelo de recomendación semántica! 🌟

Imagina que **somos** chefs en una cocina de alta tecnología musical, y **hemos** preparado una receta para encontrar las canciones que mejor se adapten a nuestros sentimientos. ¡Vamos a desglosarlo!

---

## 1. **Preparando los Ingredientes** 🥑🎶

### Limpieza del Texto: `remove_excessive_repetitions`
- **¿Qué hace?**  
  Elimina repeticiones excesivas en líneas y palabras.  
  *Ejemplo: ¡Nada de "otra vez, otra vez, otra vez!"*

- **Por qué lo usamos:**  
  **Nosotros** queremos que cada letra suene única y sin ruido. Hemos tenido que tirar a la papelera un preprocesamiento del texto excesivo 
  que hacía que el sabor final fuera bastante agridulce y fuera de nuestra finalidad .

---

### Traducción Automática: `translate_to_english`
- **¿Qué hace?**  
  Detecta el idioma de la consulta y, si no está en inglés, la traduce automáticamente. Hemos usado ya muchos scripts del estilo, finalmente aquí es donde mejor funciona.
  Hemos probado además distintos traductores que busquen algo más allá pero no mejoraban en exceso o confundían más. Así pues, si sabes inglés escribe en inglés, si no,
  a probar mezclita que no funciona nada mal y así se puede probar el producto local.

- **Por qué lo usamos:**  
  Para que **nuestro** modelo (RoBERTa) pueda entender la consulta a la perfección, sin importar el idioma de entrada.  
  ![Traducción](https://img.icons8.com/emoji/48/000000/globe-with-meridians.png)

---

### Combinación de Datos: `combined_text`
- **¿Qué hace?**  
  Une datos clave (nombre de la canción, artista, letra limpia, géneros, playlists, fecha de lanzamiento e idioma) en un solo bloque de texto.

- **Por qué lo usamos:**  
  Así, nosotros capturamos la esencia completa de cada canción en un solo vector informativo.

---

## 2. **El Secreto del Sabor: RoBERTa** 🧙‍♂️🔮

- **¿Qué hace?**  
  Genera **embeddings** de 1024 dimensiones a partir del texto combinado, convirtiendo las letras en vectores numéricos que **nuestro** sistema puede comparar.

- **Por qué lo usamos:**  
  **Nosotros** elegimos a RoBERTa porque es un auténtico maestro del lenguaje. ¡Captura el contexto y las sutilezas de cada letra como ningún otro!

---

## 3. **El Turbo Buscador: FAISS** 🚀🔍

- **¿Qué hace?**  
  Indexa los embeddings generados y permite realizar búsquedas ultrarrápidas entre miles de canciones.

- **Por qué lo usamos:**  
  Imagina tener miles de recetas y necesitar encontrar la ideal en un abrir y cerrar de ojos. **Nosotros** confiamos en FAISS para que cada búsqueda sea **rápida y precisa**.

---

## 4. **El Proceso Completo: De la Consulta al Resultado** 🔄

1. **Preparación de Datos:**  
   - Leemos el dataset y limpiamos las letras.
   - Creamos el "combo textual" uniendo toda la información relevante.

2. **Generación de Embeddings:**  
   - RoBERTa transforma el combo en vectores mágicos que encapsulan la esencia de cada canción.

3. **Indexación con FAISS:**  
   - Creamos un índice para realizar búsquedas súper rápidas.

4. **Consulta del Usuario:**  
   - Si escribes algo como "amor y desamor en la música latina", se traduce (si es necesario) y se convierte en un embedding.
   - FAISS busca en el índice y devuelve los 5 resultados más similares.

---

## 5. **Resumen Visual y Dinámico** 🎨✨

| **Herramienta**       | **Función**                                            | **¿Por qué la elegimos?**                                                 |
|-----------------------|--------------------------------------------------------|---------------------------------------------------------------------------|
| **RoBERTa**           | Transforma texto en embeddings de 1024 dimensiones     | Es el **maestro** del lenguaje que capta cada matiz de la letra.          |
| **FAISS**             | Indexa y busca rápidamente entre miles de embeddings    | Es el **turbo buscador** que encuentra resultados en un abrir y cerrar de ojos. |
| **Traducción & Detección** | Traduce y detecta el idioma de la consulta           | Para que **nuestro** sistema entienda sin importar el idioma de entrada.    |
| **Pandas, NumPy & Pickle** | Manejo y almacenamiento de datos                      | Organizan, transforman y guardan los datos como unos auténticos profesionales. |

---

## 6. **Conclusión: Nuestra Sinfonía Perfecta** 🎵✅

Hemos combinado:
- La **magia de RoBERTa** para entender el alma de cada canción
- La **velocidad de FAISS** para encontrar coincidencias al instante
- Y herramientas adicionales que aseguran que cada dato esté limpio y listo para brillar.

**Después de diversas pruebas esta es la combinación que mas nos ha gustado, por supuesto tiene mucho que mejorar
un poquito de tiempo extra para la cocción es lo que nos ha faltado.**

¡Así, cada búsqueda se convierte en una experiencia musical única y personalizada!  



In [1]:
import faiss  # Para búsquedas rápidas en el índice de embeddings
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer  # Para generar embeddings con RoBERTa
import torch  # Para usar la GPU si está disponible
import os  # Para manejar rutas de archivos
from deep_translator import GoogleTranslator  # Para traducción automática
from langdetect import detect  # Para detectar el idioma de un texto
import pickle  # Para guardar los embeddings correctamente

# =============================================================================
# 📌 Función para "sazonar" las letras y eliminar repeticiones excesivas
# =============================================================================
def remove_excessive_repetitions(text, max_reps=2):
    """
    Reduce repeticiones excesivas en el texto (líneas y palabras consecutivas).
    """
    if not isinstance(text, str):
        return text

    # 🟢 Paso 1: Eliminar repeticiones de líneas
    lines = text.splitlines()
    new_lines = []
    count = 1
    for i, line in enumerate(lines):
        if i > 0 and line.strip() == lines[i - 1].strip():
            count += 1
        else:
            count = 1
        if count <= max_reps:
            new_lines.append(line)
    text_no_line_reps = "\n".join(new_lines)

    # 🟢 Paso 2: Eliminar repeticiones de palabras consecutivas
    words = text_no_line_reps.split()
    new_words = []
    count = 1
    for i, word in enumerate(words):
        if i > 0 and word == words[i - 1]:
            count += 1
        else:
            count = 1
        if count <= max_reps:
            new_words.append(word)

    return " ".join(new_words)


# =============================================================================
# 📌 Función para traducir a inglés la consulta del usuario
# =============================================================================
def translate_to_english(text):
    """
    Traduce la consulta del usuario a inglés si no está en inglés.
    """
    try:
        detected_lang = detect(text)
        if detected_lang != 'en':
            translated_text = GoogleTranslator(source='auto', target='en').translate(text)
            print(f"🌍 Traducido: '{text}' → '{translated_text}'")
            return translated_text
        return text
    except Exception as e:
        print(f"❌ Error al detectar idioma: {e}")
        return text


# =============================================================================
# 📌 Rutas de los archivos
# =============================================================================
file_path = r"C:\Users\solan\Downloads\get_data_from_songs\data\cleaned_songs_data_v2.csv"
faiss_index_path = r"C:\Users\solan\Downloads\get_data_from_songs\src\lyrics_embeddings_faiss_IP.index"
embeddings_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\lyrics_embeddings_roberta3.pkl"

# 📌 Cargar el dataset
df = pd.read_csv(file_path)

# 📌 Aplicar la función para eliminar repeticiones en `processed_lyrics`
df['cleaned_lyrics'] = df['processed_lyrics'].apply(lambda x: remove_excessive_repetitions(x, max_reps=2))

# 📌 Crear un gran "combo textual" que contenga toda la información relevante
df['combined_text'] = (
    df['song_name'].astype(str).fillna('') + " " +
    df['artist_name'].astype(str).fillna('') + " " +
    df['cleaned_lyrics'].astype(str).fillna('') + " " +
    df['combined_genres'].astype(str).fillna('') + " " +
    df['playlists_names'].astype(str).fillna('') + " " +
    df['album_release_date'].astype(str).fillna('') + " " +
    df['language'].astype(str).fillna('')
)


# =============================================================================
# 📌 Configuración del Dispositivo y Carga del Modelo
# =============================================================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")

# 📌 Inicializar RoBERTa (Modelo para generar embeddings)
model = SentenceTransformer('sentence-transformers/all-roberta-large-v1', device=device)

# =============================================================================
# 📌 Generación de Embeddings y Creación del Índice FAISS (SIN NORMALIZACIÓN)
# =============================================================================
print("⏳ Generando nuevos embeddings (vectores de 1024 dimensiones)...")
embeddings_list = model.encode(df['combined_text'].tolist(), batch_size=32, show_progress_bar=True)

# 📌 Convertir embeddings a un array de NumPy en formato float32
embeddings_np = np.array(embeddings_list, dtype=np.float32)
d = embeddings_np.shape[1]  # Dimensión del vector (1024 en este caso)

# 📌 Crear el índice FAISS con `IndexFlatIP` (Producto Interno) sin normalización
index = faiss.IndexFlatIP(d)
index.add(embeddings_np)

# 📌 Guardar FAISS en archivo con manejo de errores
try:
    faiss.write_index(index, faiss_index_path)
    print(f"✅ FAISS `IndexFlatIP` guardado en {faiss_index_path}")
except Exception as e:
    print(f"❌ Error al guardar FAISS index: {e}")

# 📌 Guardar embeddings en Pickle para futuras consultas (🔥 SOLUCIONADO EL ERROR)
try:
    with open(embeddings_file, "wb") as f:
        pickle.dump(embeddings_list, f)  # Guardar correctamente como lista
    print(f"✅ Embeddings guardados en {embeddings_file}")
except Exception as e:
    print(f"❌ Error al guardar embeddings en Pickle: {e}")


# =============================================================================
# 📌 Ejemplo de Consulta y Búsqueda en FAISS (🔥 Ahora con Traducción)
# =============================================================================
query = "amor y desamor en la música latina"
query_translated = translate_to_english(query)
print(f"\n⏳ Procesando consulta traducida: '{query_translated}'")

# 📌 Generar el embedding de la consulta con RoBERTa
query_embedding = model.encode([query_translated], device=device, show_progress_bar=False)
query_embedding_np = np.array(query_embedding, dtype=np.float32)

# 📌 Realizar la búsqueda en FAISS (top 5 resultados)
k = 5
distances, indices = index.search(query_embedding_np, k)

# 📌 Mostrar resultados
if indices.size == 0:
    print("❌ No se encontraron resultados.")
else:
    results = []
    for i, idx in enumerate(indices[0]):
        sim_score = distances[0][i]
        song_info = df.iloc[idx].to_dict()
        song_info['similarity'] = sim_score
        results.append(song_info)

    results_df = pd.DataFrame(results)
    print("\n🔍 Resultados de la búsqueda:")
    print(results_df[['song_name', 'artist_name', 'spotify_url', 'similarity']].to_string(index=False))



⚡ Usando: CUDA
⏳ Generando nuevos embeddings (vectores de 1024 dimensiones)...


Batches:   0%|          | 0/4061 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
from langdetect import detect
import ipywidgets as widgets
from IPython.display import display, clear_output
import csv  # 🔹 Importar correctamente para manejar QUOTE_ALL

# 📌 Ruta de los archivos
embeddings_file_roberta = r"C:\Users\solan\Downloads\get_data_from_songs\src\clean_df_embeddings_roberta.pkl"
errores_file = r"C:\Users\solan\Downloads\get_data_from_songs\src\errores_canciones.csv"

# 📌 Cargar el dataset con los embeddings
df = pd.read_pickle(embeddings_file_roberta)

# 📌 Convertir embeddings a NumPy arrays (para mejor rendimiento)
df['embedding'] = df['embedding'].apply(lambda x: np.array(x))

# 📌 Cargar modelo RoBERTa en GPU si está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"⚡ Usando: {device.upper()}")

model = SentenceTransformer('all-roberta-large-v1', device=device)

def translate_to_english(text):
    """Traduce la frase del usuario a inglés si no está en inglés."""
    detected_lang = detect(text)
    if detected_lang != 'en':
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
        print(f"🌍 Traducido '{text}' ➝ '{translated_text}'")
        return translated_text
    return text

def translate_to_spanish(text):
    """Traduce una letra de inglés a español."""
    return GoogleTranslator(source='en', target='es').translate(text) if isinstance(text, str) else "Traducción no disponible"

def search_songs(user_query, top_n=5):
    """Busca canciones en base a una frase del usuario."""
    translated_query = translate_to_english(user_query)
    query_embedding = model.encode(translated_query, convert_to_tensor=True).cpu().numpy()
    df['similarity'] = df['embedding'].apply(lambda x: cosine_similarity([x], [query_embedding])[0][0])
    top_songs = df.sort_values(by="similarity", ascending=False).head(top_n).copy()
    top_songs['translated_lyrics'] = top_songs['processed_lyrics'].apply(lambda x: translate_to_spanish(x[:500]) if isinstance(x, str) else "Traducción no disponible")
    return top_songs

def mostrar_resultados(resultados):
    """Muestra las canciones en un formato legible y permite seleccionar para corregir."""
    
    for index, row in resultados.iterrows():
        print("="*80)
        print(f"🎵 **Canción:** {row['song_name']}")
        print(f"🎤 **Artista:** {row['artist_name']}")
        print(f"🔗 **Spotify:** {row['spotify_url']}")
        print(f"✅ **Similaridad:** {row['similarity']:.4f}")
        print("\n📜 **Letra en Inglés:**\n", row['processed_lyrics'][:500], "...")
        print("\n🌍 **Traducción Española:**\n", row['translated_lyrics'])
        print("="*80, "\n")
    
    seleccionar_canciones_para_correccion(resultados)

# 🔴 Variables globales necesarias para el widget
canciones_dropdown = widgets.SelectMultiple(
    options=[],
    description="Corregir:",
    layout=widgets.Layout(width="80%"),
    rows=5
)
guardar_button = widgets.Button(description="Guardar Errores", button_style="danger")
output = widgets.Output()

def seleccionar_canciones_para_correccion(resultados):
    """Permite al usuario seleccionar canciones para corregir y las guarda en errores_canciones.csv"""
    
    canciones_dropdown.options = [(f"{row['song_name']} - {row['artist_name']}", row["recording_id"]) for _, row in resultados.iterrows()]
    display(canciones_dropdown, guardar_button, output)

def guardar_errores(b):
    """Guarda las canciones seleccionadas en errores_canciones.csv"""
    with output:
        clear_output()
        seleccionados = list(canciones_dropdown.value)
        
        if not seleccionados:
            print("⚠️ No has seleccionado ninguna canción para corregir.")
            return
        
        # 📌 Filtrar las canciones seleccionadas de TODAS las columnas originales
        errores_df = df[df["recording_id"].isin(seleccionados)].copy()
        
        # 🔹 **Eliminar saltos de línea en `processed_lyrics`**
        errores_df['processed_lyrics'] = errores_df['processed_lyrics'].apply(lambda x: x.replace("\n", " ") if isinstance(x, str) else x)
        
        # 🔹 **Evitar problemas con `embedding`**
        errores_df['embedding'] = errores_df['embedding'].apply(lambda x: str(x.tolist()) if isinstance(x, np.ndarray) else x)
        
        # 📌 Guardar en CSV con comillas dobles y sin saltos de línea extra
        try:
            df_errores_previos = pd.read_csv(errores_file)
            errores_df = pd.concat([df_errores_previos, errores_df], ignore_index=True)
        except FileNotFoundError:
            pass  # Si no existe, creamos el archivo desde cero
        
        errores_df.to_csv(errores_file, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)

        print(f"✅ Se han guardado {len(seleccionados)} canciones en '{errores_file}' para corregir después.")

# 📌 Conectar el botón con la función
guardar_button.on_click(guardar_errores)

# 🔍 Prueba con una consulta
user_input = "para andar por la playa sola"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados y permitir corrección
mostrar_resultados(resultados)


⚡ Usando: CUDA
🌍 Traducido 'para andar por la playa sola' ➝ 'To walk on the beach alone'
🎵 **Canción:** nada me obliga
🎤 **Artista:** la pestilencia
🔗 **Spotify:** https://open.spotify.com/track/1ZgffyEDYfzRqr54XQKuF4
✅ **Similaridad:** 0.4243

📜 **Letra en Inglés:**
 i would like to be able to go alone without depending on anyone and make my life just go out into the world and see what i find it may be love it may be hate i dont want anything to tie me down nobody i hate passports borders and governments and if my life runs and if my life runs and if my life runs the risk that matters is my solution and if my life runs and if my life runs and if my life runs and if my life runs the risk that matters is my solution and if my life runs and if my life runs and  ...

🌍 **Traducción Española:**
 Me gustaría poder ir solo sin depender de nadie y hacer que mi vida salga al mundo y ver lo que me parece que puede ser amor, puede ser odio. gobiernos y si mi vida funciona y si mi vida funciona y

SelectMultiple(description='Corregir:', layout=Layout(width='80%'), options=(('nada me obliga - la pestilencia…

Button(button_style='danger', description='Guardar Errores', style=ButtonStyle())

Output()

In [ ]:
# 🔍 PRUEBA de velocidad con una consulta
user_input = "canciones que hablen sobre la literatura y los libros"
resultados = search_songs(user_input, top_n=5)

# 📌 Mostrar resultados de forma clara
for index, row in resultados.iterrows():
    print("\n" + "="*100)
    print(f"🎵 **Canción:** {row['song_name']}")
    print(f"🎤 **Artista:** {row['artist_name']}")
    print(f"🔗 **Spotify:** {row['spotify_url']}")
    print(f"✅ **Similaridad:** {row['similarity']:.4f}")

    # Asegurar que processed_lyrics no sea NaN y convertirlo en string
    lyrics = str(row['processed_lyrics']) if pd.notna(row['processed_lyrics']) else "Letra no disponible"
    translated_lyrics = str(row['translated_lyrics']) if pd.notna(row['translated_lyrics']) else "Traducción no disponible"

    print("\n📜 **Letra Original (Primeros 800 caracteres):**\n", lyrics[:800], "...")
    print("\n🌍 **Traducción Española (Primeros 800 caracteres):**\n", translated_lyrics[:800], "...")
    print("="*100)  


🌍 Traducido 'canciones que hablen sobre la literatura y los libros' ➝ 'songs that talk about literature and books'

🎵 **Canción:** the farmer and the viper
🎤 **Artista:** joshua powell  the great train robbery
🔗 **Spotify:** https://open.spotify.com/track/3DJWrsP2Wk1OygdnoTMfgs
✅ **Similaridad:** 0.5600

📜 **Letra Original (Primeros 800 caracteres):**
 Letra no disponible ...

🌍 **Traducción Española (Primeros 800 caracteres):**
 Traducción no disponible ...

🎵 **Canción:** these foolish things
🎤 **Artista:** etta james
🔗 **Spotify:** https://open.spotify.com/track/1AWrGMHL33h4KtehJNQNcn
✅ **Similaridad:** 0.5584

📜 **Letra Original (Primeros 800 caracteres):**
 Letra no disponible ...

🌍 **Traducción Española (Primeros 800 caracteres):**
 Traducción no disponible ...

🎵 **Canción:** cry to me
🎤 **Artista:** huey lewis  the news
🔗 **Spotify:** https://open.spotify.com/track/6yDcWkihD4DebofZLohWPn
✅ **Similaridad:** 0.5459

📜 **Letra Original (Primeros 800 caracteres):**
 Letra no dispo